In [18]:
import pygame
import heapq
import math
import random

GRID_SIZE = 20
CELL_SIZE = 30
WIDTH, HEIGHT = GRID_SIZE * CELL_SIZE, GRID_SIZE * CELL_SIZE

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
TOWER_COLOR = (0, 128, 0)
ENEMY_COLOR = (255, 0, 0)
BASE_COLOR = (0, 0, 255)
PATH_COLOR = (255, 165, 0)
HEALTH_GREEN = (0, 255, 0)
HEALTH_RED = (255, 0, 0)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Tower Defense Pathfinding with Combat")
clock = pygame.time.Clock()

grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

start_pos = (0, GRID_SIZE // 2)
base_pos = (GRID_SIZE - 1, GRID_SIZE // 2)

dirs = [(0,1),(0,-1),(1,0),(-1,0)]

class Node:
    def __init__(self, pos, parent=None):
        self.pos = pos
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.pos == other.pos
    def __lt__(self, other):
        return self.f < other.f

def heuristic(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def a_star(start, end):
    open_list = []
    closed_set = set()
    start_node = Node(start)
    end_node = Node(end)
    heapq.heappush(open_list, start_node)
    while open_list:
        current = heapq.heappop(open_list)
        closed_set.add(current.pos)
        if current == end_node:
            path = []
            while current:
                path.append(current.pos)
                current = current.parent
            return path[::-1]
        for dx, dy in dirs:
            nx, ny = current.pos[0]+dx, current.pos[1]+dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                if grid[nx][ny] == 1:
                    continue
                if (nx, ny) in closed_set:
                    continue
                neighbor = Node((nx, ny), current)
                neighbor.g = current.g + 1
                neighbor.h = heuristic(neighbor.pos, end_node.pos)
                neighbor.f = neighbor.g + neighbor.h
                skip = False
                for node in open_list:
                    if neighbor == node and neighbor.g >= node.g:
                        skip = True
                        break
                if skip:
                    continue
                heapq.heappush(open_list, neighbor)
    return None

class Enemy:
    def __init__(self, pos):
        self.pos = pos
        self.path = []
        self.path_index = 0
        self.speed = 2.5
        self.pixel_pos = (pos[1]*CELL_SIZE + CELL_SIZE//2, pos[0]*CELL_SIZE + CELL_SIZE//2)
        self.max_health = 100
        self.health = 100
        self.alive = True
    def find_path(self):
        p = a_star(self.pos, base_pos)
        if p:
            self.path = p
            self.path_index = 1
        else:
            self.path = []
            self.path_index = 0
    def update(self):
        if not self.alive:
            return
        if self.path and self.path_index < len(self.path):
            target_cell = self.path[self.path_index]
            target_px = (target_cell[1]*CELL_SIZE + CELL_SIZE//2, target_cell[0]*CELL_SIZE + CELL_SIZE//2)
            dx = target_px[0] - self.pixel_pos[0]
            dy = target_px[1] - self.pixel_pos[1]
            dist = math.hypot(dx, dy)
            if dist < self.speed:
                self.pixel_pos = target_px
                self.pos = target_cell
                self.path_index += 1
            else:
                self.pixel_pos = (self.pixel_pos[0] + self.speed*dx/dist,
                                  self.pixel_pos[1] + self.speed*dy/dist)
    def reached_base(self):
        return self.pos == base_pos
    def take_damage(self, amount):
        self.health -= amount
        if self.health <= 0:
            self.alive = False
    def draw(self, surface):
        if not self.alive:
            return
        pygame.draw.circle(surface, ENEMY_COLOR, (int(self.pixel_pos[0]), int(self.pixel_pos[1])), CELL_SIZE//3)
        health_bar_width = CELL_SIZE * 0.6
        health_bar_height = 5
        health_bar_x = self.pixel_pos[0] - health_bar_width/2
        health_bar_y = self.pixel_pos[1] - CELL_SIZE//2 - 8
        pygame.draw.rect(surface, HEALTH_RED, (health_bar_x, health_bar_y, health_bar_width, health_bar_height))
        health_ratio = max(self.health, 0) / self.max_health
        pygame.draw.rect(surface, HEALTH_GREEN, (health_bar_x, health_bar_y, health_bar_width * health_ratio, health_bar_height))

class Tower:
    def __init__(self, grid_pos):
        self.grid_pos = grid_pos
        self.range = CELL_SIZE * 4
        self.damage = 10
        self.fire_rate = 30
        self.timer = 0
        self.pixel_pos = (grid_pos[1]*CELL_SIZE + CELL_SIZE//2, grid_pos[0]*CELL_SIZE + CELL_SIZE//2)
    def update(self, enemies):
        self.timer += 1
        if self.timer < self.fire_rate:
            return None
        closest_enemy = None
        closest_dist = float('inf')
        for enemy in enemies:
            if not enemy.alive:
                continue
            ex, ey = enemy.pixel_pos
            dist = math.hypot(ex - self.pixel_pos[0], ey - self.pixel_pos[1])
            if dist <= self.range and dist < closest_dist:
                closest_enemy = enemy
                closest_dist = dist
        if closest_enemy:
            closest_enemy.take_damage(self.damage)
            self.timer = 0
            return closest_enemy
        return None
    def draw(self, surface):
        rect = pygame.Rect(self.grid_pos[1]*CELL_SIZE, self.grid_pos[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(surface, TOWER_COLOR, rect)
        pygame.draw.circle(surface, (0, 255, 255, 50), self.pixel_pos, self.range, 1)

def draw_grid():
    for r in range(GRID_SIZE):
        for c in range(GRID_SIZE):
            rect = pygame.Rect(c*CELL_SIZE, r*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            color = WHITE
            if grid[r][c] == 1:
                color = TOWER_COLOR
            pygame.draw.rect(screen, color, rect)
            pygame.draw.rect(screen, BLACK, rect, 1)

def draw_base():
    rect = pygame.Rect(base_pos[1]*CELL_SIZE, base_pos[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE)
    pygame.draw.rect(screen, BASE_COLOR, rect)

def main():
    enemies = []
    towers = []
    spawn_timer = 0
    spawn_delay = 60
    wave_number = 1
    enemies_to_spawn = 5
    enemies_spawned = 0
    running = True

    def recalc_paths():
        for e in enemies:
            if e.alive:
                e.find_path()

    while running:
        clock.tick(60)
        screen.fill(WHITE)
        draw_grid()
        draw_base()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = pygame.mouse.get_pos()
                row = my // CELL_SIZE
                col = mx // CELL_SIZE
                if event.button == 1:
                    if (row, col) != start_pos and (row, col) != base_pos:
                        if grid[row][col] == 0:
                            grid[row][col] = 1
                            towers.append(Tower((row,col)))
                        else:
                            grid[row][col] = 0
                            towers = [t for t in towers if t.grid_pos != (row,col)]
                        recalc_paths()

        if enemies_spawned < enemies_to_spawn:
            spawn_timer += 1
            if spawn_timer >= spawn_delay:
                spawn_timer = 0
                e = Enemy(start_pos)
                e.find_path()
                if e.path:
                    enemies.append(e)
                    enemies_spawned += 1

        for e in enemies[:]:
            if not e.alive:
                enemies.remove(e)
                continue
            e.update()
            if e.reached_base():
                print("An enemy reached the base! Game over or reduce base health here.")
                enemies.remove(e)

        for tower in towers:
            tower.update(enemies)

        for tower in towers:
            tower.draw(screen)

        for e in enemies:
            e.draw(screen)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


An enemy reached the base! Game over or reduce base health here.
An enemy reached the base! Game over or reduce base health here.
